#### 1. Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный).

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient, errors
from pprint import pprint
import json
import re

In [2]:
with open('secret.json', 'r') as f:
    secret = json.load(f)

In [3]:
def check_mail(login, password):

    try:

        driver = webdriver.Chrome()  
        driver.get('https://mail.ru/')
        assert 'Mail.ru: почта' in driver.title

        driver.find_element_by_id('mailbox:login').send_keys(secret['login'])
        Select(driver.find_element_by_id('mailbox:domain')).select_by_visible_text('@mail.ru')
        driver.find_element_by_id('mailbox:password').send_keys(secret['password'])
        driver.find_element_by_id('mailbox:submit').click()
        WebDriverWait(driver, 5).until(EC.title_contains('Входящие'))
        assert 'Входящие' in driver.title

        items = driver.find_elements_by_xpath("//a[contains(@class,'llc js-tooltip-direction_letter-bottom js-letter-list-item llc_normal')]")

        mails = []
        for item in items:
            link = item.get_attribute('href')
            info = item.find_element_by_class_name('llc__content')
            author = info.find_element_by_class_name('ll-crpt').get_attribute('title')
            subject = info.find_element_by_class_name('ll-sj__normal').text
            date = item.find_element_by_class_name('llc__item_date').get_attribute('title')
            mail = {'author': author, 'date': date, 'subject': subject, 'link': link}
            mails.append(mail)

        for mail in mails:
            driver.get(mail['link'])
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[contains(@id, '_BODY')]")))
            mail['text'] = driver.find_element_by_xpath("//*[contains(@id, '_BODY')]").text
            driver.back()
            WebDriverWait(driver, 5).until(EC.title_contains('Входящие'))

        driver.quit()

        return mails

    except Exception as e:
        return e


In [4]:
mails = check_mail(login=secret['login'], password=secret['password'])

In [5]:
client = MongoClient('mongodb://127.0.0.1:27017')
mail_db = client['mail']['mail']

In [6]:
mail_db.delete_many({})

for mail in mails:
    try:
        mail_db.insert_one(mail)
    except errors.DuplicateKeyError as e:
        pass

In [7]:
objects = mail_db.find({})
for obj in objects:
    pprint(obj)

{'_id': ObjectId('5d626d97d51364e207d62d37'),
 'author': 'GeekBrains <noreply@e.geekbrains.ru>',
 'date': '12 июля, 12:14',
 'link': 'https://e.mail.ru/inbox/1:36cf5bcf40155d55:0/?back=1&afterReload=1',
 'subject': '😃 Добро пожаловать! Рассказываем главное',
 'text': '        Если в письме нет картинок, жмите сюда :)\n'
         'Добро пожаловать!\n'
         'Мы — GeekBrains. Образовательный портал и отряд супергероев.\n'
         'C 2016 года — часть Mail.ru Group. Мы готовим программистов, '
         'дизайнеров, маркетологов и других спецов в Digital.\n'
         '\n'
         'С любого уровня.\n'
         '\n'
         'В чем фишка\n'
         '\n'
         'Мы НЕ учим как в университете. Ставим на максимум практики и '
         'вовлекаем в боевые проекты.\n'
         '\n'
         'Уже во время обучения вы будете создавать и запускать сайты, '
         'мобильные приложения, рекламные кампании и новые продкуты.\n'
         '\n'
         'А главное, после учебы мы поможем вам про

#### 2. Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары.

In [8]:
def get_mvideo_hits():
    
    def get_good_info(good):
        try:
            name = good.find_element_by_class_name('sel-product-tile-title').text
            link = good.find_element_by_class_name('sel-product-tile-title').get_attribute('href')
            price = float(re.sub('\D', '', good.find_element_by_class_name('c-pdp-price__current').text))
            return {'name': name, 'link': link, 'price': price}
        except:
            return {}
    
    try:
        
        driver = webdriver.Chrome()
        driver.get('https://www.mvideo.ru/')
        assert 'М.Видео' in driver.title
        
        ajax_category_carousel = driver.find_elements_by_xpath("//div[contains(@data-init, 'ajax-category-carousel')]")

        hits = None
        for element in ajax_category_carousel:
            if 'Хиты продаж' in element.text:
                hits = element
                break

        goods = []
        for _ in range(5):
            bestsellers = hits.find_elements_by_class_name('gallery-list-item')
            for bestseller in bestsellers:
                try:
                    good = get_good_info(bestseller)
                    if good != {} and good not in goods:
                        goods.append(good)
                except Exception as e:
                    pass
            button = WebDriverWait(hits, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'sel-hits-button-next'))
            )
            button.send_keys(Keys.ENTER)
        
        driver.quit()
        
        return goods
    
    except Exception as e:
        
        return e
        

In [9]:
goods = get_mvideo_hits()

In [10]:
client = MongoClient('mongodb://127.0.0.1:27017')
goods_db = client['goods']['goods']

In [11]:
goods_db.delete_many({})

for good in goods:
    try:
        goods_db.insert_one(good)
    except errors.DuplicateKeyError as e:
        pass

In [12]:
objects = goods_db.find({})
for obj in objects:
    pprint(obj)

{'_id': ObjectId('5d626da6d51364e207d62d3e'),
 'link': 'https://www.mvideo.ru/products/noutbuk-acer-aspire-3-a315-21-64a8-nx-gnver-065-30042330',
 'name': 'Ноутбук Acer Aspire 3 A315-21-64A8 NX.GNVER.065',
 'price': 19990.0}
{'_id': ObjectId('5d626da6d51364e207d62d3f'),
 'link': 'https://www.mvideo.ru/products/stiralnaya-mashina-uzkaya-hotpoint-ariston-rst-702-st-s-20032713',
 'name': 'Стиральная машина узкая Hotpoint-Ariston RST 702 ST S',
 'price': 19990.0}
{'_id': ObjectId('5d626da6d51364e207d62d40'),
 'link': 'https://www.mvideo.ru/products/smartfon-samsung-galaxy-s10e-oniks-30042525',
 'name': 'Смартфон Samsung Galaxy S10E Оникс',
 'price': 49990.0}
{'_id': ObjectId('5d626da6d51364e207d62d41'),
 'link': 'https://www.mvideo.ru/products/televizor-samsung-ue43ru7200u-10021429',
 'name': 'Телевизор Samsung UE43RU7200U',
 'price': 34990.0}
{'_id': ObjectId('5d626da6d51364e207d62d42'),
 'link': 'https://www.mvideo.ru/products/noutbuk-lenovo-ideapad-s145-15iwl-81mv00hhru-30045074',
 'nam